<a href="https://colab.research.google.com/github/jugernaut/ProgramacionEnParalelo/blob/desarrollo/Introduccion/08_Ley_Amdahl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sobre manejo de memoria




Recordemos que trabajar con *OpenMP* es trabajar bajo el modelo *PRAM* (Parallel Random Access Machine), es decir, que tenemos varios procesadores con acceso a un segmento de memoria en común, sin embargo, al trabajar varios datos a la vez hay que cuidar el manejo y acceso a la memoria. Una ventaja es que no hay límite en los números de procesadores para nuestro modelo, más que el costo económico que esto pudiera ocasionar. 

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/tree/main/Imagenes/Introduccion/MemShare.png" width="400"> 
</center>

Existen varias formas de manipular la memoria compartida, son los siguientes: 


*   EREW (**E**xclusive **R**ead **E**xclusive **W**rite)
*   CREW (**C**oncurrent **R**ead **E**xclusive **W**rite)
*   CRCW (**C**oncurrent **R**ead **C**oncurrent **W**rite)

Queda como ejercicio al lector, pensar en que problemas se deben o pueden utilizar las diferentes formas de acceder a la información.





## Taxonomía de Flynn

La taxonomía de Flynn es una clasificación para arquitecturas paralelas, la clasificación fue creada por Michael J. Flynn, la cual clasifica por la cantidad de intrucciones y flujo de datos. 

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/tree/main/Imagenes/Introduccion/Flynn.png" width="400"> 
</center>

*  SISD *(Single Instruction Single Data)*
*  MISD *(Multiple Instruction Single Data)*
*  SIMD *(Single Instruction Multiple Data)* 
*  MIMD *(Multiple Instruction Multiple Data)* 

Esta última es muy usada para explotar el paralelismo, ya sea con memoria distribuida y memoria compartida o un hibrido como son los clúsers. 



## Aceleración, eficiencia y fracción serial 

$T(n,1)$ es la complejidad en tiempo del mejor algoritmo secuencial, el número uno indica que solo se esta usando un hilo. 

$T(n,p)$ es la complejidad en tiempo del algoritmo paralelo usando $p$ procesadores. 

Existen tres métricas que nos interesan y son: la ***aceleración (Speedup)***, la ***eficiencia*** y la ***fracción serial***

Em motivo principal para usar un algoritmo paralelo es *acelerar* los cálculos secuenciales. Es por ello que nos interesa conocer la aceleración:  

$$S(p) = \frac{T(n,1)}{T(n,p)} $$

Entonces, la ***aceleración*** mide cuántas veces más rápido es el algoritmo paralelo, el valor ideal es $p$.

La ***eficiencia*** mide que tan eficientemente se están utilizanda los procesadores y el valor ideal es $1$, se calcula de la siguiente forma: 

$$E(p) = \frac{S(p)}{p} $$

Es decir, la aceleración entre el número de procesadores.

La ***fracción serial*** mide la parte del código que es inherentemente secuencial y el valor ideal es $0$.

$$ F(p) = \frac{\frac{1}{S(p)} -\frac{1}{p}}{1 - \frac{1}{p}} $$








In [ ]:
codigo = """
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <omp.h>


int main(int argc, char** argv){

    // Se valida que se haya pasado el parámetro 

	long int suma = 0;
    long int suma_p = 0; 
    long int sumaHilo = 0; 
    double inicio_s, fin_s, inicio_p, fin_p; // Para los tiempos de ejecución

    inicio_s = omp_get_wtime();
	for(int i =0; i <= 1000000000; i++){
		suma = suma +i;
	}
    fin_s = omp_get_wtime();
    double tiempo_s = fin_s - inicio_s;
	printf("suma: %li  \\n tiempo de ejecucion: %f \\n",suma, fin_s - inicio_s); //imprime resultados

    inicio_p = omp_get_wtime();

    int numHilos;
    sscanf(argv[1], "%i", &numHilos);

   	omp_set_num_threads(numHilos);
	int idHilo; 
	//inicia seccion paralela
	#pragma omp parallel private(idHilo, sumaHilo)
    {
		idHilo = omp_get_thread_num();
		if (idHilo==0){
			printf ("iniciando calculo con %i hilos\\n", omp_get_num_threads());
		}
		
		//cada hilo realiza una suma parcial
		sumaHilo=0;
		int i;
		for (i=idHilo;i<=1000000000;i+=numHilos){
			sumaHilo+=i;
		}
		
		//cada hilo actualiza la suma total con su resultado parcial
		suma_p+=sumaHilo;
	}//fin de seccion paralela


    fin_p = omp_get_wtime();
    double tiempo_p=  fin_p - inicio_p;
	printf("suma paralela: %li  \\n Tiempo de ejecucion algoritmo paralelo: %f \\n",suma,tiempo_p);
  double aceleracion = tiempo_s/tiempo_p;
  printf("La aceleración de paralelizar la suma se los primeros 1 000 000 000 número naturales es: %f \\n", aceleracion);
  double eficiencia = aceleracion/2;
  printf("La eficiencia es: %f tomando que la prueba en colab se hace en dos procesadores \\n", eficiencia);

}
""" 

In [ ]:
# se crea el archivo con permisos para escribir mediante python
archivo_texto = open("sumaIntervalo.c", "w")
# se escribe el programa en el archivo 
archivo_texto.write(codigo)
# se cierra el buffer de escritura
archivo_texto.close()

In [ ]:
!gcc -o suma -fopenmp sumaIntervalo.c

In [ ]:
!./suma 2

suma: 500000000500000000  
 tiempo de ejecucion: 3.297801 
iniciando calculo con 2 hilos
suma paralela: 500000000500000000  
 Tiempo de ejecucion algoritmo paralelo: 2.163480 
La aceleración de paralelizar la suma se los primeros 1 000 000 000 número naturales es: 1.524304 
La eficiencia es: 0.762152 tomando que la prueba en colab se hace en dos procesadores 


> Queda como ejercicio al lector calcular la fracción serial e interpretar el resultado. ⚡




In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin



¿Qué es lo que debe tomarse en cuenta cuando se hacen este tipo de pruebas? 

* Procesos en segundo plano ejecutando
* Las condiciones en las que se encuentra el equipo
* Recursos compartidos on otras aplicaciones, por ejemplo, *Mozilla*, *Spotify* entre otras...

##Ley de Amdahl 

El objetivo principal de usar un algoritmo paralelo es obtener los resultados lo más pronto posible, es decir, disminuir la complejidad en tiempo. Al aumentar el número de procesadores en el sistema paralelo se distribuyen las sub-tares entre los procesadores. Sin embargo, surge la pregunta ¿Siempre que aumente el número de procesadores disminuirá el tiempo de ejecución? 

Resulta que la ley de Amdahl dice que llega un punto en el cual sin importar que el numero de procesadores sea muy alto , el speedup se va a comportar de manera lineal. Dicho de otra forma cada sección paralelizable de un algoritmo tendrá un número $n$ de procesadores optimos, si ejecutamos el algoritmo con más de $n$ procesadores no mejorará el tiempo de ejecución.



<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/tree/main/Imagenes/Introduccion/LeyAmdahl.png" width="400"> 
</center>

La gráfica anterior nos muestra que dependiendo del porcentaje de código ejecutado de manera paralela será la cota de número de procesadores para alcanzar la máximo eficiencia. 

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/tree/main/Imagenes/Introduccion/LeyATime.png" width="400"> 
</center>

### La ley de Amdahl en suma paralela

En el siguiente cuadro de datos tenemos algunas ejecuciones de pruebas realizadas en una computadora con las siguientes carácteristicas, se obtuvo un tiempo de ejecución de: 3 255 980 microsegundos

* Marca *Dell*
* Intel(R) Core(TM) i5-6300U CPU 2.40 GHz 
* 4 CPU's  
* 16 Gb de RAM

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/tree/main/Imagenes/Introduccion/SumaParalelaLDA.png" width="400">
</center>

Observemos que en el Cuadro el mejor tiempo ($1107980{.}8 \mu s$) que se presenta es cuando el programa usa únicamente 4 hilos, la computadora donde se ejecutaron las pruebas es una Dell con 4 CPU(s). En el cuadro tenemos que los mejores tiempos obtenidos corresponden al renglón de las pruebas realizadas con 50 hilos. Incluso podemos observar que los mejores tiempos corresponden a las pruebas realizadas con 50 hilos. Recordemos que cualquier problema tiene una cota en tiempo óptimo, además también hay una cota para el número de procesadores que pueden optimizar un proceso. Cada problema tendrá un número de hilos óptimo, para este caso tenemos que 50 hilos nos proporciona el tiempo óptimo, al usar 100 aumentamos el tiempo en vez de disminuirlo, se tendría que buscar el número $n$ donde se pierde la optimización del problema. 